# Juan Carlos Perez Ramirez
## Procesamiento de Lenguaje Natural
## Proyecto final

### Librerias

In [1]:
import torch
import pickle
from scrn import SCRNModel

2025-05-27 09:29:41.373528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-27 09:29:41.373786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-27 09:29:41.469712: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-27 09:29:41.635307: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-27 09:29:42.707677: W tensorflow/compiler/tf2

### Carga del modelo entrenado

In [3]:
with open("vocab.pkl", "rb") as f:
    char2idx = pickle.load(f)
idx2char = {idx: ch for ch, idx in char2idx.items()}

vocab_size = len(char2idx)

# Crear modelo con mismos parámetros usados al entrenar
model = SCRNModel(
    vocab_size=vocab_size,
    input_size=64,
    hidden_size=64,
    context_size=40,
    alpha=0.95
)
model.load_state_dict(torch.load("scrn_model.pt"))
model.eval()

SCRNModel(
  (embedding): Embedding(48, 64)
  (rnn_cell): SCRNCell(
    (linear): Linear(in_features=168, out_features=64, bias=True)
  )
  (fc): Linear(in_features=64, out_features=48, bias=True)
)

### Generacion de texto

In [4]:
def generate_text(model, char2idx, idx2char, prompt, length=200, device="cpu"):
    model.to(device)
    model.eval()

    input_size = model.embedding.embedding_dim
    hidden_size = model.rnn_cell.hidden_size
    context_size = model.rnn_cell.context_size

    # Estado inicial
    h_t = torch.zeros(1, hidden_size).to(device)
    s_t = torch.zeros(1, context_size).to(device)

    # Codifica prompt
    generated = list(prompt)
    input_seq = [char2idx[c] for c in prompt if c in char2idx]
    
    for idx in input_seq[:-1]:
        x = torch.tensor([[idx]]).to(device)
        embedded = model.embedding(x).squeeze(1)  # (1, input_size)
        h_t, (h_t, s_t) = model.rnn_cell(embedded, (h_t, s_t))

    current_idx = input_seq[-1] if input_seq else torch.randint(0, vocab_size, (1,)).item()

    for _ in range(length):
        x = torch.tensor([[current_idx]]).to(device)
        embedded = model.embedding(x).squeeze(1)
        h_t, (h_t, s_t) = model.rnn_cell(embedded, (h_t, s_t))
        logits = model.fc(h_t)
        probs = torch.softmax(logits, dim=1)

        # Elegir el siguiente carácter (puedes usar sample o argmax)
        current_idx = torch.multinomial(probs, num_samples=1).item()
        generated.append(idx2char[current_idx])

    return "".join(generated)


In [6]:
prompt = "the meaning of life is"
output = generate_text(model, char2idx, idx2char, prompt, length=300)
print(output)


the meaning of life is $ n million manume a sharet dust been mond welderal obor and dexoposs <unk> bes 
 to manur havel 
 the clief 
 fundle curroductures are n n n n to of some of that these 's sendcy davled that new to ferning 
 sist 
 all controm 
 fram ewrsicy report econditblas seell 
 ther n <unk> eq said but 
 whi


In [8]:
prompt = "the first day of the week"
output = generate_text(model, char2idx, idx2char, prompt, length=300)
print(output)


the first day of the week <unk> 
 geargh many the concerdor researge said that equiriss of <unk> as <unk> in new n n $ n freacaust was sidead 
 manage ary tust <unk> <unk> custs onled at most time the some 
 the kny echan pendan 
 but the aboulasinespite including weoks about into defell degulatures devely dochanged <unk> b


In [9]:
prompt = "after many days of travel"
output = generate_text(model, char2idx, idx2char, prompt, length=300)
print(output)


after many days of travely <unk> to that it company because on <unk> some shil what sovity polen 
 the than taker part 
 the polity bone 
 a conton <unk> addinated <unk> <unk> said <unk> <unk> counting win hong <unk> <unk> yestons drupereatment doplow for neldon the fill 
 the have considents and in a <unk> 
 the recited fa


In [6]:
prompt = "It is generally believed that using a strong nonlinearity is necessary to capture complex"
output = generate_text(model, char2idx, idx2char, prompt, length=300)
print(output)

It is generally believed that using a strong nonlinearity is necessary to capture complexo so. he be a <unk> fordafs 
 the dival funds land instill that alsomerly desponsay yith. naym 
 japach concrucing this some to said would day a call mexue 
 to corpanage sell bliddat to will long that worced 
 gond law less which sersiday that is a franting scomicial azal decaled lound 
 subjecttic
